# HERE POI Shapefile Decoding

In [5]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib, loadJoblib
from geoUtils import *
from geospatialUtils import saveGeoData, getBB
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-12-14 11:20:41.503466


In [9]:
# Global Params
basedir="/Users/tgadf/Downloads/here"
prec=7
mainkey="HEREEXTPOI"

# Load HERE POI Data

In [3]:
from pandas import DataFrame
from os.path import join
import pickle
here_filename = join(basedir, "HEREdata.p")
if not isFile(here_filename):
    raise ValueError("HERE data {0} does not exist".format(here_filename))
print("Loading HERE POI data from {0}".format(here_filename))
here_data = pickle.load(open(here_filename, "rb"))
here_data['lat'] = here_data['lat']/1e5
here_data['lon'] = here_data['lon']/1e5
use_here_data = here_data[here_data['iso_country_code'] == "USA"]

Loading HERE POI data from /Users/tgadf/Downloads/here/HEREdata.p


In [4]:
fastfood=["ConvenienceStore", "CoffeeShop"]
restaurant=["Restaurant"]
homefood=["GroceryStore", "Pharmacy"]
lodging=["Hotel","SkiResort"]
commercial=["ClothingStore","ConsumerElectronicsStore","Bookstore", "DepartmentStore", "Shopping", 'OfficeSupplyandServiceStore', 'HomeSpecialtyStore']
commercial+=['SportingGoodStore','SpecialtyStore', "HomeImprovementandHardwareStore", "Bank", "ATM"]
auto=['AutoDealerships', 'AutomobileClub', 'AutoServiceandMaintenance', 'RentalCarAgency', 'TruckDealership', 'MotorcycleDealership']
fuel=['PetrolandGasolineStation']
cycle=['BicycleParking', 'BicycleService', 'BicycleSharingLocation']
municipal=["Library", "PostOffice", "PoliceStation", "CityHall","CivicandCommunityCentre", "ConventionandExhibitionCentre", "CourtHouse"]
industrial=["CargoCentre", "BusinessFacility"]
building=["Embassy", 'MilitaryBase']
medical=["Hospital"]
attraction=["AmusementPark", "AnimalPark", "Museum", "TouristAttraction", "TouristInformation", "Winery","PerformingArts","HistoricalMonument"]
sport=["GolfCourse"]
entertainment=["Cinema", "Casino"]
transit=['CommuterRailStation', 'BusStation', 'FerryTerminal', 'TransportationService', 'TrainStation', 'Airport', 'TaxiStand', 'HighwayExit', "WeighStation"]
recreation=["SportsComplex", "Trailhead", "Dock", "Marina", "Campground","ParkandRecreationArea"]
school=["School"]
college=["HigherEducation"]
parking=['ParkandRide', 'ParkingGarageandHouse', 'ParkingLot', 'RestArea', 'TruckStopandPlaza', 'TruckParking', 'BorderCrossing', 'DeliveryEntrance']

# Create Individual HERE Data

In [5]:
shapeData   = {}
geoShapeMap = {}
ngeos = {}
shapeval = 'here'
Nshapes = 0
catIDs_filename = join(basedir, "catIDs.p")
if not isFile(catIDs_filename):
    raise ValueError("Categery IDs data {0} does not exist".format(catIDs_filename))
    
catIDs = pickle.load(open(catIDs_filename, "rb"))
for row,rowdata in use_here_data.iterrows():
    if Nshapes % 250000 == 0 and Nshapes > 0:
        print(row,Nshapes)
    Nshapes += 1
    irec   = row
    catID  = str(int(rowdata['cat_id']))
    geoid  = catID
    name   = catIDs.get(catID)
    #geoid  = rowdata['poi_id']
    #htype  = rowdata['cat_id']
    
    fclass = name
    if fclass in fastfood:
        fclass = "fastfood"
        size = 15
    elif fclass in restaurant:
        fclass = "restaurant"
        size = 20
    elif fclass in homefood:
        fclass = "grocery"
        size = 50
    elif fclass in lodging:
        fclass = "lodging"
        size = 50
    elif fclass in commercial:
        fclass = "commercial"
        size = 25
    elif fclass in auto:
        fclass = "auto"
        size = 25
    elif fclass in municipal:
        fclass = "municipal"
        size = 50
    elif fclass in industrial:
        fclass = "industrial"
        size = 50
    elif fclass in building:
        fclass = "building"
        size = 10
    elif fclass in medical:
        fclass = "medical"
        size = 50
    elif fclass in attraction:
        fclass = "attraction"
        size = 100
    elif fclass in sport:
        fclass = "sport"
        size = 500
    elif fclass in entertainment:
        fclass = "entertainment"
        size = 50
    elif fclass in recreation:
        fclass = "recreation"
        size = 50
    elif fclass in school:
        fclass = "school"
        size = 50
    elif fclass in college:
        fclass = "college"
        size = 100
    elif fclass in cycle:
        fclass = "cycling"
        size = 10
    elif fclass in transit:
        fclass = "transit"
        size = 10
    elif fclass in parking:
        fclass = "parking"
        size = 25
    elif fclass in fuel:
        fclass = "fuel"
        size = 10
    else:
        raise ValueError("fClass {0} was not found!".format(fclass))    
    
    catID = fclass
    geoid = fclass
    name  = fclass
    
    if name is None:
        print(rowdata)
        1/0
    if shapeData.get(catID) is None:
        shapeData[catID] = {}
        shapeData[catID][geoid]   = {"Name": name}
        geoShapeMap[catID] = {}
        geoShapeMap[catID][geoid] = set()

    lat0 = rowdata['lat']
    lng0 = rowdata['lon']
    
    dLat = convertMetersToLat(size)
    dLng = convertMetersToLong(size, lat0)
    
    for i in [1, 0, -1]:
        for j in [1, 0, -1]:
            lat = lat0 + i*dLat
            lng = lng0 + j*dLng
            
            geo = geohash.encode(latitude=lat, longitude=lng, precision=prec)
            geoShapeMap[catID][geoid].add(geo)
    
for catID,catData in shapeData.items():
    shapeval = catID
    ngeos = len(geoShapeMap[catID][catID])
    print("Found {0} of type {1}".format(ngeos,shapeval))
    saveGeoData(shapeData[catID], geoShapeMap[catID], Nshapes, ngeos, join(basedir, "{0}-{1}".format(shapeval, prec)))

402753 250000
805836 500000
1209512 750000
1613642 1000000
2017407 1250000
2420441 1500000
2824037 1750000
3227998 2000000
3631825 2250000
4034628 2500000
Found 440822 of type auto


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/here/auto-7-data.p


There are 440822 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/here/auto-7-geos.p


Found 495288 of type restaurant


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/here/restaurant-7-data.p


There are 495288 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/here/restaurant-7-geos.p


Found 309715 of type grocery


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/here/grocery-7-data.p


There are 309715 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/here/grocery-7-geos.p


Found 699913 of type commercial


There are 1 entries in the saved file.
Saved shape data to /Users/t

# Combine HERE Data Into Standard Format

In [6]:
vals    = glob(join(basedir, "*-{0}-geos.p".format(prec)))
vals    = [splitext(basename(x))[0].split('-')[0] for x in vals]
geomap  = {}
records = {}
for mtype in vals:
    try:
        recdata = pickle.load(open(join(basedir, "{0}-{1}-data.p".format(mtype, prec)), "rb"))
        geodata = pickle.load(open(join(basedir, "{0}-{1}-geos.p".format(mtype, prec)), "rb"))
    except:
        continue
    ngeos = 0
    for geoid,geos in geodata.items():
        ngeos += len(geos)
        for geo in geos:
            if geomap.get(geo) is None:
                geomap[geo] = Counter()
            geomap[geo][mtype] += 1

    for geoid,rec in recdata.items():
        records[geoid] = rec['Name']
        break
                
    print("{0}\t{1}\t---> {2} <---".format(mtype,ngeos,len(geomap)))


for geo in geomap.keys():
    geomap[geo] = dict(geomap[geo])
    
fname=join(basedir, "geomap-{0}-{1}.p".format(prec, mainkey))
print("Writing {0}".format(fname))
saveJoblib(geomap, fname)
showSize(fname)

fname=join(basedir, "georec-{0}-{1}.p".format(prec, mainkey))
print("Writing {0}".format(fname))
saveJoblib(records, fname)
showSize(fname)

attraction	200756	---> 200756 <---
auto	440822	---> 626136 <---
building	2456	---> 628154 <---
college	35933	---> 657964 <---
commercial	699913	---> 1172966 <---
cycling	343	---> 1173037 <---
entertainment	21177	---> 1181415 <---
fastfood	191917	---> 1258975 <---
fuel	146945	---> 1283622 <---
grocery	309715	---> 1405389 <---
industrial	39532	---> 1431516 <---
lodging	187737	---> 1544305 <---
medical	20095	---> 1559287 <---
municipal	190530	---> 1663846 <---
parking	48844	---> 1683361 <---
recreation	291819	---> 1915730 <---
restaurant	495288	---> 2023961 <---
school	359680	---> 2298670 <---
sport	139539	---> 2430004 <---
transit	80371	---> 2490656 <---
Writing /Users/tgadf/Downloads/here/geomap-7-HEREPOI.p
  --> This file is 27.9MB.
  --> This file is 27.9MB.
Writing /Users/tgadf/Downloads/here/georec-7-HEREPOI.p
  --> This file is 238B.
  --> This file is 238B.


# Folium Plots (if needed)

In [13]:
# Load (if needed)
fname=join(basedir, "geomap-{0}-{1}.p".format(prec, mainkey))
geodata = loadJoblib(fname)
geos = geodata.keys()
print("There are {0} geos".format(len(geos)))

In [14]:
from folium import PolyLine, CircleMarker, Circle, Marker, Icon, FeatureGroup, Map, LayerControl
import geohash

def addCircles(geos, m, init=None, maxDist=None, debug=False):
    cols = ['darkgreen']

    toPlot = []
    for geo in geos:
        com = geohash.decode_exactly(geo)[:2]
        if init is not None:
            if getDist(com, init) < maxDist:
                toPlot = getBB(geo, istuple=True)
                popup  = ", ".join(geodata[geo].keys())
                PolyLine(toPlot, color='blue', weight=1, popup=popup, opacity=1).add_to(m)
        else:
            Circle(com, color=cols[0], radius=rad, fill=True, fill_color=cols[0], weight=1, opacity=0).add_to(m)
        
def createMap(init=None):
    if init is not None:
        m = Map(location=[init[0], init[1]], zoom_start=10)
    else:
        m = Map(location=[40, -95], zoom_start=4)
        
    return m        

In [18]:
init=(35.733663732, -84.342998628)
init=(36.02193, -84.23355)
m = createMap(init)
addCircles(geos, m=m, init=init, maxDist=2500)
m.save("/Users/tgadf/Downloads/here/here-map.html")